In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

### 读取CSV文件

In [2]:
path = 'E:/Source/ThsisSource/DataSource/CICIDS2017/data/'
save_path = 'E:/Source/ThsisSource/DataSource/CICIDS2017/process/'
csv_file =['Friday-WorkingHours-Afternoon-DDos.pcap_ISCX','Friday-WorkingHours-Morning.pcap_ISCX',
'Monday-WorkingHours.pcap_ISCX','Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX',
'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX','Tuesday-WorkingHours.pcap_ISCX','Wednesday-workingHours.pcap_ISCX']
end = '.csv'
data = pd.read_csv(path+csv_file[0]+end)
# 循环添加数据集
for i in range(1,7):
    tmp_data=pd.read_csv(path+csv_file[i]+end)
    data=pd.concat([data,tmp_data],axis=0,ignore_index=True)
data

,Destination Port,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,Fwd Packet Length Mean,Fwd Packet Length Std,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,54865,3,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,55054,109,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,55055,52,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,46236,34,1,1,6,6,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,54863,3,2,0,12,0,6,6,6.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2544271,53,32215,4,2,112,152,28,28,28.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2544272,53,324,2,2,84,362,42,42,42.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2544273,58030,82,2,1,31,6,31,0,15.5,21.92031,...,32,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2544274,53,1048635,6,2,192,256,32,32,32.0,0.00000,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


In [3]:
# 获取特征行
data.keys()

Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

### 对数据集中的缺失值补0

In [4]:
data = data.fillna(value=0)

### 对数据集中Flow Packets/s Flow Bytes/s的无穷大值进行修改

In [5]:
for i in range(len(data[' Flow Packets/s'])):
    if data[' Flow Packets/s'][i] == float('inf'):
        data.loc[i, ' Flow Packets/s'] = '2000001'
for i in range(len(data['Flow Bytes/s'])):
    if data['Flow Bytes/s'][i] == float('inf'):
        data.loc[i, 'Flow Bytes/s'] = '1040000001'

### 将Label改成数字类型

In [6]:
Label = data[' Label']
Label = np.array(Label)
# 删除标签列
data_ = data.drop([' Label'], axis=1)
data = np.array(data_)
# 修改
i = 0
for i in range(Label.shape[0]):
    if Label[i] == "BENIGN":
        Label[i] = 0
    else:
        Label[i] = 1
Label

array([0, 0, 0, ..., 0, 0, 0], dtype=object)

### 层次划分数据集

In [7]:
#为了缩小数据集同时不改变少数类的样本过少
train_x,test_x,train_y,test_y = train_test_split(data,Label,test_size=0.01,random_state=50,stratify=Label)
pd.value_counts(pd.DataFrame(test_y)[0])

0    21456
1     3987
Name: 0, dtype: int64

### 数据标准化和数据降维


In [8]:
# 标准化降维
X = test_x.astype(float)
Y = test_y.reshape(-1,1)
# 标准化
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
# 降维
pca = PCA(n_components=0.9)
pca.fit(X_scaled)
X_pca = pca.transform(X_scaled)
# # 拼接
X_pca = np.array(X_pca)
X2 = np.concatenate((X_pca, Y), axis=1)
print(X2.shape)
data2 = pd.DataFrame(X2)
data2

(25443, 19)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,-1.970958,0.106996,0.108828,-0.57245,1.185896,-0.778337,-0.546489,0.660487,0.279376,0.043361,-0.012038,-0.230153,0.318785,0.025449,-0.355501,0.381098,-0.434769,0.330443,0
1,-1.975041,0.105816,0.128128,-0.527378,1.449879,-1.036373,-0.600377,0.901814,0.362956,0.061245,-0.022983,-0.366235,0.320579,0.04383,-0.370865,0.365739,-0.450702,0.252928,0
2,-1.903837,0.104508,0.087018,-0.542264,1.158707,-0.695709,-0.600175,0.48396,0.175489,0.051417,-0.040394,0.00585,0.013933,0.054785,0.273085,-0.653553,0.391065,-0.236325,0
3,-1.418707,0.054825,0.030445,-0.041844,0.466516,1.499424,1.0139,-2.24115,-0.269697,-0.171197,0.264839,-0.669452,0.242975,-0.205633,0.285894,1.524059,1.609524,0.488502,0
4,6.97082,-1.086115,12.696349,-1.26539,2.922475,-2.559051,5.441502,0.209097,-0.309525,-0.089806,-0.744672,3.745463,-0.353493,-2.817231,0.508798,-0.190335,0.183545,-0.461095,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25438,-0.728763,0.161038,-1.421186,1.857293,1.049338,0.911539,1.881652,-2.497266,0.522252,-0.19996,-0.199329,-1.061518,0.82854,-0.096188,0.43969,1.674747,-0.289983,0.767537,0
25439,-2.16878,0.119322,0.312787,0.201621,-1.668705,0.447266,-0.164863,-0.695624,-1.997928,-0.172788,0.861298,0.568783,-0.392367,-0.53342,-0.135973,0.794104,0.931116,-0.005776,0
25440,-1.927106,0.1013,0.068224,-0.656997,1.530189,-1.062294,-0.61546,0.961403,0.395601,0.066445,-0.030081,-0.381431,0.343162,0.013001,-0.365415,0.301419,-0.51826,0.298327,0
25441,-1.466232,0.030386,0.326278,-0.690867,0.030161,0.239705,-1.57757,-0.931264,0.723815,-0.003875,-0.370533,0.866998,-0.045199,1.525975,2.426771,-1.431149,0.456146,-0.354912,0


### 保存文件到对应路径

In [9]:
data2.to_csv(save_path+'result_CICIDS2017_2checkpoint'+end, encoding='utf-8')